In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 03 - Analisi Risultati\n",
    "\n",
    "Notebook per analizzare i risultati salvati dopo il processing.\n",
    "\n",
    "**Obiettivi:**\n",
    "- Caricare risultati salvati\n",
    "- Visualizzare traiettorie e pose\n",
    "- Analisi statistica dei dati\n",
    "- Export dati per ulteriori analisi"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.insert(0, '..')\n",
    "\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from pathlib import Path\n",
    "import pandas as pd\n",
    "\n",
    "from src.utils.data_io import load_tracked_points, load_pose\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carica Risultati"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "VIDEO_NAME = 'video1'  # MODIFICA\n",
    "RESULTS_DIR = Path('../data/results')\n",
    "\n",
    "# Carica punti tracciati\n",
    "points_file = RESULTS_DIR / 'tracked_points' / f'{VIDEO_NAME}_points.npz'\n",
    "tracked_points, metadata = load_tracked_points(str(points_file))\n",
    "\n",
    "print(f\"✓ Punti tracciati caricati: {tracked_points.shape}\")\n",
    "print(f\"  Frames: {len(tracked_points)}\")\n",
    "print(f\"  Punti per frame: {tracked_points.shape[1]}\")\n",
    "if metadata:\n",
    "    print(f\"  Metadata: {metadata}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Carica pose\n",
    "poses_file = RESULTS_DIR / 'poses' / f'{VIDEO_NAME}_poses.npz'\n",
    "data = np.load(poses_file)\n",
    "rotations = data['rotations']\n",
    "translations = data['translations']\n",
    "\n",
    "print(f\"✓ Pose caricate: {len(rotations)} frames\")\n",
    "print(f\"  Rotations shape: {rotations.shape}\")\n",
    "print(f\"  Translations shape: {translations.shape}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Analisi Punti Tracciati"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calcola centro tra i due fari\n",
    "centers = tracked_points.mean(axis=1)\n",
    "\n",
    "# Plot traiettoria 2D\n",
    "plt.figure(figsize=(14, 10))\n",
    "plt.plot(centers[:, 0], centers[:, 1], 'b-', linewidth=2, label='Centro veicolo')\n",
    "plt.plot(centers[0, 0], centers[0, 1], 'go', markersize=15, label='Inizio')\n",
    "plt.plot(centers[-1, 0], centers[-1, 1], 'ro', markersize=15, label='Fine')\n",
    "\n",
    "# Plot fari individuali\n",
    "plt.plot(tracked_points[:, 0, 0], tracked_points[:, 0, 1], \n",
    "         'c--', alpha=0.5, linewidth=1, label='Faro sinistro')\n",
    "plt.plot(tracked_points[:, 1, 0], tracked_points[:, 1, 1], \n",
    "         'm--', alpha=0.5, linewidth=1, label='Faro destro')\n",
    "\n",
    "plt.xlabel('X (pixel)', fontsize=12)\n",
    "plt.ylabel('Y (pixel)', fontsize=12)\n",
    "plt.title(f'Traiettoria 2D - {VIDEO_NAME}', fontsize=14)\n",
    "plt.legend(fontsize=10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.gca().invert_yaxis()\n",
    "plt.show()\n",
    "\n",
    "# Statistiche\n",
    "print(\"\\nStatistiche Traiettoria 2D:\")\n",
    "print(f\"  Range X: [{centers[:, 0].min():.1f}, {centers[:, 0].max():.1f}] px\")\n",
    "print(f\"  Range Y: [{centers[:, 1].min():.1f}, {centers[:, 1].max():.1f}] px\")\n",
    "print(f\"  Movimento X: {centers[:, 0].max() - centers[:, 0].min():.1f} px\")\n",
    "print(f\"  Movimento Y: {centers[:, 1].max() - centers[:, 1].min():.1f} px\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analisi distanza tra fari (dovrebbe rimanere costante)\n",
    "distances = np.linalg.norm(tracked_points[:, 0] - tracked_points[:, 1], axis=1)\n",
    "\n",
    "plt.figure(figsize=(12, 5))\n",
    "plt.plot(distances, 'b-', linewidth=2)\n",
    "plt.axhline(distances.mean(), color='r', linestyle='--', \n",
    "            label=f'Media: {distances.mean():.1f} px')\n",
    "plt.xlabel('Frame', fontsize=12)\n",
    "plt.ylabel('Distanza (pixel)', fontsize=12)\n",
    "plt.title('Distanza tra Fari nel Tempo', fontsize=14)\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nStatistiche Distanza Fari:\")\n",
    "print(f\"  Media: {distances.mean():.1f} px\")\n",
    "print(f\"  Std Dev: {distances.std():.1f} px\")\n",
    "print(f\"  Range: [{distances.min():.1f}, {distances.max():.1f}] px\")\n",
    "print(f\"  Variazione: {distances.std() / distances.mean() * 100:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Analisi Pose 3D"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estrai posizioni 3D\n",
    "positions = translations.reshape(-1, 3)\n",
    "\n",
    "# Calcola distanze\n",
    "distances_3d = np.linalg.norm(positions, axis=1)\n",
    "\n",
    "# Plot distanza nel tempo\n",
    "fps = metadata.get('fps', 30.0) if metadata else 30.0\n",
    "time = np.arange(len(distances_3d)) / fps\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(time, distances_3d, 'b-', linewidth=2)\n",
    "plt.axhline(distances_3d.mean(), color='r', linestyle='--',\n",
    "            label=f'Media: {distances_3d.mean():.2f}m')\n",
    "plt.xlabel('Tempo (s)', fontsize=12)\n",
    "plt.ylabel('Distanza (m)', fontsize=12)\n",
    "plt.title(f'Distanza dal Veicolo - {VIDEO_NAME}', fontsize=14)\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nStatistiche Distanza 3D:\")\n",
    "print(f\"  Media: {distances_3d.mean():.2f}m\")\n",
    "print(f\"  Std Dev: {distances_3d.std():.2f}m\")\n",
    "print(f\"  Range: [{distances_3d.min():.2f}, {distances_3d.max():.2f}]m\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot posizione 3D\n",
    "fig = plt.figure(figsize=(14, 10))\n",
    "ax = fig.add_subplot(111, projection='3d')\n",
    "\n",
    "# Traiettoria\n",
    "ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], \n",
    "        'b-', linewidth=2, label='Traiettoria')\n",
    "\n",
    "# Punti inizio/fine\n",
    "ax.scatter(positions[0, 0], positions[0, 1], positions[0, 2],\n",
    "              c='g', marker='o', s=200, label='Inizio')\n",
    "ax.scatter(positions[-1, 0], positions[-1, 1], positions[-1, 2],\n",
    "              c='r', marker='o', s=200, label='Fine')\n",
    "\n",
    "ax.set_xlabel('X (m)', fontsize=12)\n",
    "ax.set_ylabel('Y (m)', fontsize=12)\n",
    "ax.set_zlabel('Z (m)', fontsize=12)\n",
    "ax.set_title(f'Posizione 3D del Veicolo - {VIDEO_NAME}', fontsize=14)\n",
    "ax.legend()\n",
    "ax.grid(True, alpha=0.3)\n",
    "\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nStatistiche Posizione 3D:\")\n",
    "print(f\"  X range: [{positions[:, 0].min():.2f}, {positions[:, 0].max():.2f}]m\")\n",
    "print(f\"  Y range: [{positions[:, 1].min():.2f}, {positions[:, 1].max():.2f}]m\")\n",
    "print(f\"  Z range: [{positions[:, 2].min():.2f}, {positions[:, 2].max():.2f}]m\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Converti rotazioni in angoli di Eulero\n",
    "import cv2\n",
    "\n",
    "yaws = []\n",
    "pitches = []\n",
    "rolls = []\n",
    "\n",
    "for rvec in rotations:\n",
    "    R, _ = cv2.Rodrigues(rvec)\n",
    "    \n",
    "    sy = np.sqrt(R[0, 0]**2 + R[1, 0]**2)\n",
    "    singular = sy < 1e-6\n",
    "    \n",
    "    if not singular:\n",
    "        roll = np.arctan2(R[2, 1], R[2, 2])\n",
    "        pitch = np.arctan2(-R[2, 0], sy)\n",
    "        yaw = np.arctan2(R[1, 0], R[0, 0])\n",
    "    else:\n",
    "        roll = np.arctan2(-R[1, 2], R[1, 1])\n",
    "        pitch = np.arctan2(-R[2, 0], sy)\n",
    "        yaw = 0\n",
    "    \n",
    "    rolls.append(np.degrees(roll))\n",
    "    pitches.append(np.degrees(pitch))\n",
    "    yaws.append(np.degrees(yaw))\n",
    "\n",
    "# Plot orientamento\n",
    "fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)\n",
    "\n",
    "axes[0].plot(time, yaws, 'b-', linewidth=2)\n",
    "axes[0].set_ylabel('Yaw (°)', fontsize=12)\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "axes[0].set_title(f'Orientamento Veicolo - {VIDEO_NAME}', fontsize=14)\n",
    "\n",
    "axes[1].plot(time, pitches, 'g-', linewidth=2)\n",
    "axes[1].set_ylabel('Pitch (°)', fontsize=12)\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "axes[2].plot(time, rolls, 'r-', linewidth=2)\n",
    "axes[2].set_ylabel('Roll (°)', fontsize=12)\n",
    "axes[2].set_xlabel('Tempo (s)', fontsize=12)\n",
    "axes[2].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nStatistiche Orientamento:\")\n",
    "print(f\"  Yaw   - Media: {np.mean(yaws):.1f}°, Range: [{np.min(yaws):.1f}, {np.max(yaws):.1f}]°\")\n",
    "print(f\"  Pitch - Media: {np.mean(pitches):.1f}°, Range: [{np.min(pitches):.1f}, {np.max(pitches):.1f}]°\")\n",
    "print(f\"  Roll  - Media: {np.mean(rolls):.1f}°, Range: [{np.min(rolls):.1f}, {np.max(rolls):.1f}]°\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Export Dati per Analisi Esterna"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Crea DataFrame con tutti i dati\n",
    "df = pd.DataFrame({\n",
    "    'frame': np.arange(len(positions)),\n",
    "    'time_s': time,\n",
    "    'pos_x_m': positions[:, 0],\n",
    "    'pos_y_m': positions[:, 1],\n",
    "    'pos_z_m': positions[:, 2],\n",
    "    'distance_m': distances_3d,\n",
    "    'yaw_deg': yaws,\n",
    "    'pitch_deg': pitches,\n",
    "    'roll_deg': rolls,\n",
    "    'center_x_px': centers[:, 0],\n",
    "    'center_y_px': centers[:, 1]\n",
    "})\n",
    "\n",
    "# Salva CSV\n",
    "csv_path = RESULTS_DIR / f'{VIDEO_NAME}_analysis.csv'\n",
    "df.to_csv(csv_path, index=False)\n",
    "\n",
    "print(f\"✓ Dati esportati: {csv_path}\")\n",
    "print(f\"\\nPrime righe:\")\n",
    "print(df.head())\n",
    "\n",
    "print(f\"\\nStatistiche:\")\n",
    "print(df.describe())"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}